In [1]:
import requests as rq
import bs4 as bs
import json
import traceback
import re
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import glob, os

## 1. BBC

In [4]:
links_to_visit = {}

In [5]:
def get_link(url):
    home_bbc = rq.get(url)
    soup = bs.BeautifulSoup(home_bbc.text, 'html.parser')
    for a in soup.find_all('a', href=True):
        if a.text:
            link = a["href"]
            if link.startswith("/swahili/habari-") or link.startswith("/swahili/michezo-"):
                if link not in list(links_to_visit.keys()):
                    links_to_visit[link] = False

In [6]:
get_link("https://www.bbc.com/swahili")
get_link("https://www.bbc.com/swahili/michezo")
len(links_to_visit)

47

In [7]:
for link in list(links_to_visit.keys()):
    link_url = "https://www.bbc.com/" + link
    if links_to_visit[link] == False:
        get_link(link_url)
        links_to_visit[link] = True
        
        no_link = all(value == True for value in list(links_to_visit.values()))
        if no_link:
            break

In [8]:
len(list(links_to_visit))

104

## 2. RFI

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [2]:
links_visit_rfi = {}
load_more = "btn-load-more alt more-loader"

seed = ["http://sw.rfi.fr/ulaya/all/", 
        "http://sw.rfi.fr/amerika/all/", 
        "http://sw.rfi.fr/mashariki-ya-kati/all/",
        "http://sw.rfi.fr/asia/all/",
        "http://sw.rfi.fr/afrika/all/",
        "http://sw.rfi.fr/eac/all/",
        "http://sw.rfi.fr/siasa-uchumi/all/",
        "http://sw.rfi.fr/afya-mazingira/all/",
        "http://sw.rfi.fr/utamaduni/all/",
        "http://sw.rfi.fr/michezo/all/"]

driver = webdriver.Chrome(executable_path="/home/aims/Downloads/chromedriver")
driver.get("http://sw.rfi.fr/afrika/all/")
loadmore = driver.find_element_by_class_name("cookie-bar_hide")
loadmore.click()
while True:
    try:
        time.sleep(1)
        loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "btn-load-more")))
        loadmore.click()
        time.sleep(1)
    except Exception:
        print("Reached bottom of page")
        traceback.print_exc()
        break

soup_rfi = bs.BeautifulSoup(driver.page_source,'html.parser')
# home_rfi.text
# soup_rfi = bs.BeautifulSoup(home_rfi.text, 'html.parser')
news_links = soup_rfi.find_all('a', href=True)
len(news_links)

Reached bottom of page


Traceback (most recent call last):
  File "<ipython-input-2-e97f09742b74>", line 22, in <module>
    loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "btn-load-more")))
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/wait.py", line 71, in until
    value = method(self._driver)
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/expected_conditions.py", line 128, in __call__
    return _element_if_visible(_find_element(driver, self.locator))
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/expected_conditions.py", line 415, in _find_element
    raise e
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/expected_conditions.py", line 411, in _find_element
    return driver.find_element(*by)
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 978, in find_element
    'value': v

WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.88)


In [96]:
def save_data(data, filename="final.txt"):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s" % item)

In [74]:
news_final = []

for a in news_links:
    if a.text:
        link = a["href"]
        matching = re.match(r'^\/michezo\/\d+\w+', link)
        if matching:
            new_link = 'http://sw.rfi.fr'+link
            news_final.append(new_link)

save_data(news_final, 'michezo.txt')

In [97]:
os.chdir("./")
final_data = []
for file in glob.glob("*.txt"):
    with open(file) as f:
        for line in f:
            if line not in final_data:
                final_data.append(line)

In [98]:
save_data(final_data, 'news_final.txt')
len(final_data)

19593

In [3]:
urls = []
def get_urls():
    with open("news_final.txt") as f:
        for line in f:
            urls.append(line)

In [4]:
get_urls()

In [5]:
import asyncio
import concurrent.futures
import requests

final_data = []

def do_req(url):
    return requests.get(url, headers=headers)

async def main():

    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                do_req,
                url
            )
            for url in urls
        ]
        for response in await asyncio.gather(*futures):
            page_soup = bs.BeautifulSoup(response.text, 'html.parser')
            news_title = page_soup.select_one('article h1')
            news_title = news_title.text if news_title else ''
            news_body = page_soup.select_one('article div[itemprop="articleBody"]')
            news_body = news_body.text if news_body else ''
            final_data.append({'title' : news_title, 'content' : news_body})


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [10]:
full = []
for item in final_data:
    if item['content'] != '':
        full.append(item)

In [11]:
with open('data_full.json', 'w') as fout:
    json.dump(full, fout)

## 3. VOA

### 3.1 Swahili

In [37]:
home_voa_swahili = rq.get("https://www.voaswahili.com/")

In [38]:
home_voa_swahili

<Response [200]>

### 3.2 Lingala

In [3]:
# home_voa_lingala = rq.get("https://www.voalingala.com") / https://www.voalingala.com/z/5744

driver = webdriver.Chrome(executable_path="/home/aims/Downloads/chromedriver")
driver.get("https://www.voalingala.com/z/5744")
while True:
    try:
        time.sleep(1)
        loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "link-showMore")))
        loadmore.click()
        time.sleep(1)
    except Exception:
        print("Reached bottom of page")
        traceback.print_exc()
        break

Reached bottom of page


Traceback (most recent call last):
  File "<ipython-input-3-e7bb62ea74ad>", line 8, in <module>
    loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "link-showMore")))
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



In [6]:
home_page = bs.BeautifulSoup(driver.page_source,'html.parser')
news_links = home_page.find_all('a', href=True)
len(news_links)

2062

In [7]:
with open("data.json", "r") as read_file:
    data = json.load(read_file)

In [12]:
with open('sw_final.txt', 'w') as file:
    for item in data:
        join_title_description = item['title'] + '\n\n' + item['content'] + '\n\n'
        file.write(join_title_description)